In [1]:
import cobra
import cobra.manipulation
from cobra.io import load_model

In [2]:
model = load_model('textbook') #loads the test model, 'textbook' is E. Coli's core metabolism

In [3]:
print(len(model.reactions)) #prints number of reactions, metabolites, and genes, as objects within the cobra.Dictlist (which is 'model' in this case)
print(len(model.metabolites))
print(len(model.genes))

95
72
137


In [4]:
model

Name,e_coli_core
Memory address,7f427c54e810
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


In [5]:
model.reactions[29] #gets 30th reaction in the model

Reaction identifier,EX_glu__L_e
Name,L-Glutamate exchange
Memory address,0x7f420f3b01d0
Stoichiometry,glu__L_e --> L-Glutamate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
model.metabolites.get_by_id('atp_c') #retrieves items by id. 


Metabolite identifier,atp_c
Name,ATP
Memory address,0x7f420f480fb0
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"GLNS, PPS, Biomass_Ecoli_core, GLNabc, ACKr, ATPM, PGK, SUCOAS, ADK1, ATPS4r, PYK, PFK, PPCK"


### Reactions

In [7]:
pgi = model.reactions.get_by_id('PGI') #gives table of information on the enzyme and reaction
print(pgi.name) #just gives the name
print(pgi.reaction) #just gives the reaction
print(pgi.lower_bound, pgi.upper_bound) #lower bound is less than zero indicates a reversible reaction. Bounds are the constraints applied to the flux of a biochemical reaction, defining the minimum and maximum allowable rates the reaction can occur
print(pgi.reversibility)
pgi.bounds = (-1000.0, 1000.0) #creates new bounds for the reaction 


glucose-6-phosphate isomerase
g6p_c <=> f6p_c
-1000.0 1000.0
True


In [8]:
pgi.check_mass_balance() #checks if the reaction is balanced. Returns an empty bracket if there are no errors.  

{}

In [9]:
pgi.add_metabolites({model.metabolites.get_by_id('h_c'): -1}) #adds a metabolite in, with a given charge 
print(pgi.reaction)
print(pgi.check_mass_balance()) #reaction after addition is not balanced

g6p_c + h_c <=> f6p_c
{'charge': -1.0, 'H': -1.0}


In [10]:
pgi.subtract_metabolites({model.metabolites.get_by_id('h_c'): -1}) #removes the metabolite and rebalances the reaction 
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


### Metabolites

In [11]:
atp = model.metabolites.get_by_id("atp_c") #retrieves atp metabolite. In this case, 
atp

Metabolite identifier,atp_c
Name,ATP
Memory address,0x7f420f480fb0
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"GLNS, PPS, Biomass_Ecoli_core, GLNabc, ACKr, ATPM, PGK, SUCOAS, ADK1, ATPS4r, PYK, PFK, PPCK"


In [12]:
print(atp.name)
print(atp.compartment) #the compartment in this case is where the metabolite is located, c = cytosol 
print(atp.charge)
print(atp.formula)
print(atp.reactions) #all the reactions ATP is involved in

ATP
c
-4
C10H12N5O13P3
frozenset({<Reaction GLNS at 0x7f420f3b3800>, <Reaction PPS at 0x7f420f35fa10>, <Reaction Biomass_Ecoli_core at 0x7f420f40c230>, <Reaction GLNabc at 0x7f420f3b3e30>, <Reaction ACKr at 0x7f420fcfec90>, <Reaction ATPM at 0x7f420f40c8c0>, <Reaction PGK at 0x7f420f35e2d0>, <Reaction SUCOAS at 0x7f420f1494c0>, <Reaction ADK1 at 0x7f420f3e3b00>, <Reaction ATPS4r at 0x7f420f40cd10>, <Reaction PYK at 0x7f420f148380>, <Reaction PFK at 0x7f420f35d3a0>, <Reaction PPCK at 0x7f420f35f5c0>})


### Genes

In [13]:
gpr = pgi.gpr #gives the gene-protein-reaction relationship. In this case, the gene is b4025
print(gpr)
gpr_string = pgi.gene_reaction_rule #this gives the gene as a string. Might be useful....
print(gpr_string)
pgi_gene = model.genes.get_by_id('b4025')
pgi_gene.reactions #reaction asociated with b4025

b4025
b4025


frozenset({<Reaction PGI at 0x7f420f35df10>})

In [14]:
pgi.gene_reaction_rule = "(spam or eggs)" #changes the gene reaction rule. This syntax is necessary. 
print(pgi.genes)
pgi_gene.reactions #shows there's no reaction associated with the new gene

frozenset({<Gene eggs at 0x7f420f3b1e80>, <Gene spam at 0x7f427c579010>})


frozenset()

In [15]:
model.genes.get_by_id('spam')

Gene identifier,spam
Name,
Memory address,0x7f427c579010
Functional,True
In 1 reaction(s),PGI


### Gene Knockouts

In [16]:
cobra.manipulation.knock_out_model_genes(model, ["spam"]) #knocks out the gene 'spam'
print("after 1 KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound)) #prints the new bounds of the flux (rates of reactions in model) after knockout

cobra.manipulation.knock_out_model_genes(model, ["eggs"])
print("after 2 KO:  %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound)) #zero flux means the reaction is not occuring. 

after 1 KO: -1000 < flux_PGI < 1000
after 2 KO:     0 < flux_PGI <    0


In [17]:
model = load_model('textbook') #resets the model to the original
for reaction in model.reactions[:5]: #A normal for loop, saying for each item in the list of reactions from 0 to 4, 
    with model as model: #creates a context manager -> temporarily allows for manipulating the model without changing it permenantly, i.e. the model goes back to normal after the block is executed. 
        reaction.knock_out() #knocks out each reaction 
        model.optimize() #
        print('%s blocked (bounds: %s), new growth rate %f' % #all these strange percentages are placeholders for the variables given in the next line. %s is a string, %f is a float
            (reaction.id, str(reaction.bounds), model.objective.value)) #objective function in this case is the growth rate of the model. It can be changed
    


ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


In [18]:
[reaction.bounds for reaction in model.reactions[:5]] #all model bounds have been reverted back to normal

[(-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0)]

In [19]:
print(model.objective.expression) #shows the objective function of the model
#1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba means maximize (1) the flux through the biomass_Ecoli_core reaction and minimize (-1) the flux through the biomass core reverse reaction

1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


In [20]:
with model: #context manager
    model.objective = 'ATPM' #sets the objective function to the reaction with id ATPM, which optimizes the flux through the ATP maintenance reaction
    print('print objective in first context:', model.objective.expression) 
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)
print('back to original objective:', model.objective.expression)

#What this does is show an example of changing contexts within a model. One could change the context of the objective function, change the reaction, see the output, and do that sequentially

print objective in first context: 1.0*ATPM - 1.0*ATPM_reverse_5b752
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: 1.0*ATPM - 1.0*ATPM_reverse_5b752
back to original objective: 1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


In [21]:
atpm = model.reactions.get_by_id('ATPM')
atpm

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x7f420f2974a0
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP + H2O --> ADP + H+ + Phosphate
GPR,
Lower bound,8.39
Upper bound,1000.0
